## 4. WAV 파일 형식 검사 (손상 여부 & 샘플링 레이트 확인)

In [1]:
pip install librosa soundfile

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import librosa
import soundfile as sf
import pandas as pd

# 검사할 WAV 파일이 위치한 상위 디렉토리
data_dir = "/home/ubuntu/data"

# 파일 정보를 저장할 리스트
file_info_list = []

# 검사할 파일 확장자
audio_extensions = [".wav"]

# WAV 파일 검사
for root, _, files in os.walk(data_dir):
    for file in files:
        if any(file.endswith(ext) for ext in audio_extensions):
            file_path = os.path.join(root, file)
            try:
                # 파일 로드하여 정보 추출
                with sf.SoundFile(file_path) as f:
                    sample_rate = f.samplerate
                    channels = f.channels
                    subtype = f.subtype_info  # 비트 깊이 등 형식 정보
                    duration = len(f) / sample_rate  # 파일 길이 (초)
                
                # 오류 메시지 없음
                error_msg = None

            except Exception as e:
                # 파일 로드 중 오류 발생 시 정보 기록
                sample_rate = None
                channels = None
                subtype = None
                duration = None
                error_msg = str(e)

            # 파일 정보 저장
            file_info_list.append({
                "file_path": file_path,
                "file_name": file,
                "sample_rate": sample_rate,
                "channels": channels,
                "bit_depth": subtype,
                "duration": duration,
                "error_msg": error_msg
            })

# 데이터프레임으로 변환
df = pd.DataFrame(file_info_list)

# 전체 파일 개수
total_files = len(df)

# 손상된 파일 개수
corrupt_files = df[df["error_msg"].notnull()]
num_corrupt = len(corrupt_files)

# 샘플링 레이트 통계
sample_rate_stats = df["sample_rate"].value_counts(dropna=False)

# 비트 깊이 통계
bit_depth_stats = df["bit_depth"].value_counts(dropna=False)

# 채널 수 통계
channel_stats = df["channels"].value_counts(dropna=False)

# 결과 출력
print(f"📊 총 파일 개수: {total_files}")
print(f"❌ 손상된 파일 개수: {num_corrupt}")
print("\n🔢 샘플링 레이트 통계:")
print(sample_rate_stats)
print("\n🎚️ 비트 깊이 통계:")
print(bit_depth_stats)
print("\n🔈 채널 수 통계:")
print(channel_stats)

# 손상된 파일 목록 출력
if num_corrupt > 0:
    print("\n❌ 손상된 파일 목록 및 오류 메시지:")
    for idx, row in corrupt_files.iterrows():
        print(f"파일: {row['file_name']}, 오류: {row['error_msg']}")

# 파일별 상세 정보 CSV로 저장 (선택사항)
# df.to_csv("audio_file_info.csv", index=False)

📊 총 파일 개수: 18048
❌ 손상된 파일 개수: 0

🔢 샘플링 레이트 통계:
sample_rate
44100    17779
48000      173
22050       71
32000       25
Name: count, dtype: int64

🎚️ 비트 깊이 통계:
bit_depth
Signed 16 bit PCM    17080
Signed 32 bit PCM      968
Name: count, dtype: int64

🔈 채널 수 통계:
channels
1    16297
2     1751
Name: count, dtype: int64
